In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from sklearn.metrics import accuracy_score
import numpy as np

In [2]:
train_dir = 'C:/Users/it shop/Downloads/Telegram Desktop/Alzheimer_s Dataset two/Alzheimer_s Dataset/train'
test_dir = 'C:/Users/it shop/Downloads/Telegram Desktop/Alzheimer_s Dataset two/Alzheimer_s Dataset/test'


In [3]:
img_size = (128, 128)
batch_size = 32

In [4]:
datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    subset='training',
    class_mode='categorical'
)

Found 4098 images belonging to 4 classes.


In [5]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    subset='validation',
    class_mode='categorical'
)

Found 254 images belonging to 4 classes.


In [6]:
# Build the CNN Feature Extractor
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
])

cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

C:\Users\it shop\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
# Extract Features Using CNN
features_train = cnn_model.predict(train_generator)
features_test = cnn_model.predict(test_generator)

C:\Users\it shop\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


129/129 ━━━━━━━━━━━━━━━━━━━━ 132s 1s/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step


In [8]:
# Save the CNN model
cnn_model.save('cnn_feature_extractor.h5')

In [9]:
# Build and Train the ANN Model
ann_model = Sequential([
    Input(shape=(features_train.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(4, activation='softmax')  # Adjust the number of classes
])

ann_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
# Convert labels to one-hot encoding
train_labels = np.array(train_generator.labels)
test_labels = np.array(test_generator.labels)
train_labels_one_hot = tf.keras.utils.to_categorical(train_labels, num_classes=4)
test_labels_one_hot = tf.keras.utils.to_categorical(test_labels, num_classes=4)


In [11]:
# Train the ANN model
ann_model.fit(features_train, train_labels_one_hot, validation_data=(features_test, test_labels_one_hot), epochs=10)

Epoch 1/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - accuracy: 0.4859 - loss: 1.1214 - val_accuracy: 0.5039 - val_loss: 1.0266
Epoch 2/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.4721 - loss: 1.0475 - val_accuracy: 0.5039 - val_loss: 1.0254
Epoch 3/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.4923 - loss: 1.0379 - val_accuracy: 0.5039 - val_loss: 1.0288
Epoch 4/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.4920 - loss: 1.0503 - val_accuracy: 0.5039 - val_loss: 1.0284
Epoch 5/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.5015 - loss: 1.0333 - val_accuracy: 0.5039 - val_loss: 1.0250
Epoch 6/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.4905 - loss: 1.0477 - val_accuracy: 0.5039 - val_loss: 1.0275
Epoch 7/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.4883 - loss: 1.0471 - val_accuracy: 0.5039 - val_loss: 1.0240
Epoch 8/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.5014 - loss: 1.0465 - val_accurac

In [12]:
# Calculate accuracy
predictions = np.argmax(ann_model.predict(features_test), axis=1)
test_accuracy = accuracy_score(test_labels, predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Test Accuracy: 50.39%


In [13]:
# Save the ANN model
ann_model.save('ann_model.h5')

In [15]:
#Classify New Images
def classify_image(img_path):
    # Load the saved CNN and ANN models
    cnn_model = tf.keras.models.load_model('cnn_feature_extractor.h5')
    ann_model = tf.keras.models.load_model('ann_model.h5')

    # Load and preprocess the new image
    img = image.load_img(img_path, target_size=img_size)  # Resize to match CNN input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize

    # Extract features using the CNN model
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  # Flatten for ANN

    # Classify the features with the ANN model
    prediction = ann_model.predict(features)
    predicted_class = np.argmax(prediction, axis=1)  # Get the class with the highest probability

    # Interpret the prediction
    class_labels = {0: 'MildDemented', 1: 'ModerateDemented', 2: 'NonDemented', 3: 'VeryMildDemented'}
    result = class_labels[predicted_class[0]]
    return result

# Test the function with a new image
img_path = r'C:/Users/it shop/Downloads/Telegram Desktop/Alzheimer_s Dataset two/Alzheimer_s Dataset/test/NonDemented/28 (29).jpg'
result = classify_image(img_path)
print(f"The image is classified as: {result}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 459ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
The image is classified as: NonDemented
